In [1]:
import pandas as pd
import plotly.express as px
from statsmodels.tsa.seasonal import seasonal_decompose
import logging
import datetime
import time

import dash
from dash import dcc, html
from dash.dependencies import Input, Output

import calendar
import pandas as pd
import plotly.express as px
from pyspark.sql import SparkSession
from pyspark.sql.functions import concat, col
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose
from ipywidgets import interact, widgets
from matplotlib.dates import MonthLocator, DateFormatter
import geopandas as gpd
from shapely.geometry import Point
from sklearn.preprocessing import StandardScaler
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import geopandas as gpd
from mpl_toolkits.axes_grid1 import make_axes_locatable
#from fbprophet import Prophet
from sklearn.metrics import mean_squared_error
import numpy as np
import plotly.graph_objects as go
from pyspark.sql import functions as F
from pyspark.sql.functions import year, month
from sklearn.metrics.pairwise import euclidean_distances
from pyspark.sql.functions import year, month, sum as sum_
from scipy.spatial.distance import euclidean
from prophet import Prophet
from pyspark.sql.functions import col, countDistinct , desc , count , length
from pyspark.sql.functions import split, col, when
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf, dayofmonth, last_day ,to_date, lit, isnull
from pyspark.sql.types import IntegerType
import numpy as np
from sklearn.metrics import r2_score, mean_squared_error
from dateutil.relativedelta import relativedelta
from pyspark.sql.functions import date_format

import schedule
from pyspark.sql.functions import col, to_timestamp, date_format

import pyspark
import pandas as pd
import boto3
import sagemaker
import sagemaker.feature_store.feature_store as fs
import databricks.connect
from dateutil.relativedelta import relativedelta

import pandas as pd
import numpy as np
import plotly.graph_objects as go
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, mean_absolute_error
from sklearn.metrics import mean_squared_error, root_mean_squared_error

import pandas as pd
import numpy as np
from prophet import Prophet
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error
import plotly.graph_objects as go
from itertools import product
import random
import statsmodels.api as sm
import os

import logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
from databricks.connect import DatabricksSession

# Get spark
spark = databricks.connect.DatabricksSession.builder.getOrCreate()

c:\Users\gusta\AppData\Local\pypoetry\Cache\virtualenvs\ltv-novo-aF9Dy8BA-py3.10\lib\site-packages\pydantic\_internal\_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[02/07/25 15:43:38] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=442105;file://c:\Users\gusta\AppData\Local\pypoetry\Cache\virtualenvs\ltv-novo-aF9Dy8BA-py3.10\lib\site-packages\botocore\credentials.py\credentials.py]8;;\:]8;id=811419;file://c:\Users\gusta\AppData\Local\pypoetry\Cache\virtualenvs\ltv-novo-aF9Dy8BA-py3.10\lib\site-packages\botocore\credentials.py#1278\1278]8;;\

sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\gusta\AppData\Local\sagemaker\sagemaker\config.yaml


## Tabela Nova

In [2]:
query=  """WITH 
clientes AS 
(
    SELECT 
          COUNT(1)              AS contratos
        , cgcmf                 AS documento
        , MIN(data_cad)         AS data_inicio
        , MAX(bloqueio)         AS data_final
    FROM 
    (
        SELECT DISTINCT
            matriz, codigo, fis_jurid, cgcmf, raz_social, max_conexao, bloqueio,
            sit_cliente, data_cad, vendedor, vlrfatmin, vlr_dispacesso, receita
        FROM 
            prd.dl_redeok_checkok.tb_cliente cli
        WHERE 
            cli.sit_cliente IN ('A', 'F', 'N', 'P', 'D')
            AND (cli.matriz = cli.codigo OR falsa = 's')
            AND cli.receita IN (9, 10, 46, 58)
            AND cli.cgcmf IS NOT NULL
            AND cli.cgcmf != '23236392000198'
            AND LENGTH(TRIM(cli.cgcmf)) = 14
            AND cli.cgcmf RLIKE '^[0-9]+$'
            AND NOT EXISTS 
            (
                SELECT 1 
                FROM prd.dl_redeok_checkok.tb_revenda rev 
                WHERE cli.codigo = rev.codigo
            )
    ) 
    GROUP BY cgcmf
),
titulo_cobranca AS 
(
    SELECT 
          documento         AS documento
        , COUNT(meses)      AS faturas
    FROM 
    (
        SELECT DISTINCT
            cli.cgcmf                           AS documento
            , DATE_TRUNC('month', rec.emissao)  AS meses
        FROM 
            prd.dl_redeok_checkok.vw_receber rec
        INNER JOIN prd.dl_redeok_checkok.tb_cliente cli 
            ON cli.codigo = rec.cliente
        WHERE 
            rec.situacao IN ('A', 'B')
            AND NOT EXISTS 
            (
                SELECT 1 
                FROM prd.dl_redeok_checkok.tb_revenda rev 
                WHERE rev.codigo = rec.cliente
            )
            AND cli.sit_cliente IN ('A', 'F', 'N', 'P', 'D')
            AND (cli.matriz = cli.codigo OR falsa = 's')
            AND cli.receita IN (9, 10, 46, 58)
            AND cli.cgcmf IS NOT NULL
            AND cli.cgcmf != '23236392000198'
            AND LENGTH(TRIM(cli.cgcmf)) = 14    
            AND cli.cgcmf RLIKE '^[0-9]+$'
    ) 
    GROUP BY 
        documento
),
ultimo_boleto AS 
(
    SELECT 
        cli.cgcmf AS documento,
        MAX(rec.emissao) AS ultima_emissao
    FROM 
        prd.dl_redeok_checkok.vw_receber rec
    INNER JOIN prd.dl_redeok_checkok.tb_cliente cli 
        ON cli.codigo = rec.cliente
    WHERE 
        rec.situacao IN ('A', 'B')
        AND NOT EXISTS 
        (
            SELECT 1 
            FROM prd.dl_redeok_checkok.tb_revenda rev 
            WHERE rev.codigo = rec.cliente
        )
        AND cli.sit_cliente IN ('A', 'F', 'N', 'P', 'D')
        AND (cli.matriz = cli.codigo OR falsa = 's')
        AND cli.receita IN (9, 10, 46, 58)
        AND cli.cgcmf IS NOT NULL
        AND cli.cgcmf != '23236392000198'
        AND LENGTH(TRIM(cli.cgcmf)) = 14    
        AND cli.cgcmf RLIKE '^[0-9]+$'
    GROUP BY cli.cgcmf
)
SELECT 
      cli.contratos
    , cli.documento
    , cli.data_inicio
    , cli.data_final
    , CASE 
          WHEN cli.data_final IS NULL THEN NULL
          ELSE ult.ultima_emissao
      END AS data_ultimo_boleto
      , tit.faturas
    , ROUND(MONTHS_BETWEEN(COALESCE(cli.data_final, CURRENT_DATE()), cli.data_inicio)) AS meses_contrato 
FROM 
    clientes cli
INNER JOIN titulo_cobranca tit 
    ON cli.documento = tit.documento
LEFT JOIN ultimo_boleto ult
    ON cli.documento = ult.documento;



"""


# Executando a consulta e obtendo os dados em um DataFrame Spark
sdf_final= spark.sql(query)


In [3]:
sdf_final.show()
sdf_final.count()

+---------+--------------+--------------------+--------------------+--------------------+-------+--------------+
|contratos|     documento|         data_inicio|          data_final|  data_ultimo_boleto|faturas|meses_contrato|
+---------+--------------+--------------------+--------------------+--------------------+-------+--------------+
|        1|67902890000106|2003-08-21 09:36:...|                NULL|                NULL|    231|         258.0|
|        1|07047708000147|2014-01-13 00:00:...|2015-05-05 00:00:...|2015-06-06 00:00:...|     13|          16.0|
|        1|08327876000159|2011-03-31 00:00:...|2014-08-28 00:00:...| 2014-08-05 00:00:00|     38|          41.0|
|        1|00194945000191|2008-03-12 00:00:...|2024-08-23 16:24:...|2024-08-01 00:00:...|    193|         197.0|
|        1|01574674000162|2007-03-26 00:00:...|2008-12-02 00:00:...| 2008-11-01 00:00:00|     20|          20.0|
|        1|02944466000170|2012-10-30 00:00:...|2025-01-27 08:30:...|2025-02-01 00:00:...|    147

134317

## Tabela Churn

In [4]:
churn_sdf = spark.table("dev.dw_visualization.tb_vw_churn")
churn_sdf.show()
churn_sdf.count()

+--------------------+-------------------+-------------------+-----------------+---------------------+---------------------------------+---------------------------+---------------+--------------------+--------------------+-----------------+-------------+----------------------+--------------------------------+-------------------------------+--------------------------------------+-------------------------+-----------------------------------+-------------------------+-------------------------------+-----------------------------------------+----------------------+---------------------+-----------------------+----------------------------+---------------------------------+---------------------+----------------+--------------------+--------------------+--------------------+--------------+-----------+--------------+------------+----------------------+--------------+------------------------+---------------------------------+--------------------+--------------------+------------------+----------

28485

## Transformação em anos

In [5]:
sdf_final = sdf_final.withColumn("faturas_anos", col("faturas") / 12)
sdf_final = sdf_final.withColumn("meses_contrato_anos", col("meses_contrato") / 12)

## Estatisicas da tabela nova

In [6]:
from pyspark.sql.functions import col

num_clientes = sdf_final.count()

print(f"Quantidade de clientes : {num_clientes}")

# Contar valores nulos
num_nulos = sdf_final.filter(col("data_final").isNull()).count()

# Contar valores não nulos
num_nao_nulos = sdf_final.filter(col("data_final").isNotNull()).count()

print(f"Quantidade de clientes ativos: {num_nulos}")
print(f"Quantidade de clientes Finalizados: {num_nao_nulos}")

# Filtrando os registros onde 'meses_contrato_anos' é negativo
clientes_com_meses_negativos = sdf_final.filter(col("meses_contrato_anos") < 0)

# Contando a quantidade de clientes
quantidade_clientes = clientes_com_meses_negativos.count()

# Exibindo o resultado
print(f"Quantidade de clientes com 'meses_contrato_anos' negativo: {quantidade_clientes}")


# Contar quantos registros têm data_final menor que data_inicio
quantidade_datas_invalidas = sdf_final.filter(col("data_final") < col("data_inicio")).count()

print(f"Quantidade de clientes com 'data_final' menor que 'data_inicio': {quantidade_datas_invalidas}")



Quantidade de clientes : 134317
Quantidade de clientes ativos: 13283
Quantidade de clientes Finalizados: 121034
Quantidade de clientes com 'meses_contrato_anos' negativo: 1168
Quantidade de clientes com 'data_final' menor que 'data_inicio': 1177


## Clientes Ativos e Finalizados

In [7]:
# Filtrar e manter apenas registros com 'meses_contrato_anos' maior que zero
sdf_final_filtro = sdf_final.filter(col("meses_contrato_anos") >= 0)

# Contar a quantidade de clientes após o filtro
num_clientes_filtrados = sdf_final_filtro.count()
print(f"Quantidade de clientes após remoção de 'meses_contrato_anos' menores ou iguais a zero: {num_clientes_filtrados}")

Quantidade de clientes após remoção de 'meses_contrato_anos' menores ou iguais a zero: 133149


In [8]:
# Contar valores nulos
sdf_ativos = sdf_final_filtro.filter(col("data_final").isNull())

# Contar valores não nulos
sdf_cancelados = sdf_final_filtro.filter(col("data_final").isNotNull())

In [9]:
sdf_ativos.show()
sdf_ativos.count()

+---------+--------------+--------------------+----------+------------------+-------+--------------+------------------+-------------------+
|contratos|     documento|         data_inicio|data_final|data_ultimo_boleto|faturas|meses_contrato|      faturas_anos|meses_contrato_anos|
+---------+--------------+--------------------+----------+------------------+-------+--------------+------------------+-------------------+
|        1|67902890000106|2003-08-21 09:36:...|      NULL|              NULL|    231|         258.0|             19.25|               21.5|
|        1|19710352000169|2020-03-16 12:17:...|      NULL|              NULL|     59|          59.0| 4.916666666666667|  4.916666666666667|
|        1|08585965000103|2008-03-28 00:00:...|      NULL|              NULL|    139|         202.0|11.583333333333334| 16.833333333333332|
|        1|07841484000140|2009-09-08 00:00:...|      NULL|              NULL|    179|         185.0|14.916666666666666| 15.416666666666666|
|        1|094758380

13308

In [10]:
sdf_cancelados.show()
sdf_cancelados.count()

+---------+--------------+--------------------+--------------------+--------------------+-------+--------------+------------------+-------------------+
|contratos|     documento|         data_inicio|          data_final|  data_ultimo_boleto|faturas|meses_contrato|      faturas_anos|meses_contrato_anos|
+---------+--------------+--------------------+--------------------+--------------------+-------+--------------+------------------+-------------------+
|        1|07047708000147|2014-01-13 00:00:...|2015-05-05 00:00:...|2015-06-06 00:00:...|     13|          16.0|1.0833333333333333| 1.3333333333333333|
|        1|08327876000159|2011-03-31 00:00:...|2014-08-28 00:00:...| 2014-08-05 00:00:00|     38|          41.0|3.1666666666666665| 3.4166666666666665|
|        1|00194945000191|2008-03-12 00:00:...|2024-08-23 16:24:...|2024-08-01 00:00:...|    193|         197.0|16.083333333333332| 16.416666666666668|
|        1|01574674000162|2007-03-26 00:00:...|2008-12-02 00:00:...| 2008-11-01 00:00:00

119841

## Numero de Contratos por Cliente

In [11]:
import plotly.graph_objects as go

# Converter sdf_final para Pandas
contratos_pd = sdf_final.select("contratos").toPandas()

# Contar a frequência de cada número de contratos
contratos_freq = contratos_pd["contratos"].value_counts().reset_index()
contratos_freq.columns = ["num_contratos", "frequencia"]

# Criar gráfico de barras
fig = go.Figure()

fig.add_trace(go.Bar(
    x=contratos_freq["num_contratos"], 
    y=contratos_freq["frequencia"],
    text=contratos_freq["frequencia"],  # Adiciona os valores acima das barras
    textposition='outside'  # Posiciona os rótulos fora das barras
))

# Configurar layout para melhorar a visualização
fig.update_layout(
    title="Distribuição do Número de Contratos por Cliente",
    xaxis_title="Número de Contratos por Cliente",
    yaxis_title="Frequência",
    bargap=0.2,
    height=600,  # Aumenta o tamanho do gráfico
    yaxis_type="log",  # Escala logarítmica para visualizar valores pequenos e grandes
)

# Exibir o gráfico
fig.show()


## Tempo de Permanencia - Filtrado - Faturas

In [12]:
import plotly.graph_objects as go
import pandas as pd

# Converter para Pandas para visualização
df_permanencia = sdf_final_filtro.select("faturas_anos", "data_final").toPandas()

# Garantir que há valores para definir o bin máximo
max_anos = df_permanencia["faturas_anos"].max()
bins = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, max_anos + 1]  # Adiciona um espaço extra para o último bin
labels = [
    "0-1 ano",
    "1-2 anos",
    "2-3 anos",
    "3-4 anos",
    "4-5 anos",
    "5-6 anos",
    "6-7 anos",
    "7-8 anos",
    "8-9 anos",
    "9-10 anos",
    "10+ anos"
]

# Criar coluna categórica com os intervalos corrigidos
df_permanencia["faixa_permanencia"] = pd.cut(df_permanencia["faturas_anos"], bins=bins, labels=labels, right=False)

# Classificar contratos como Ativo (nulo) ou Cancelado (não nulo)
df_permanencia["status_contrato"] = df_permanencia["data_final"].apply(lambda x: "Ativo" if pd.isna(x) else "Cancelado")

# Contar a quantidade de contratos ativos e cancelados por faixa de permanência
df_faixas_status = df_permanencia.groupby(["faixa_permanencia", "status_contrato"]).size().unstack(fill_value=0)

# Criar gráfico de barras agrupadas (um do lado do outro)
fig = go.Figure()

# Adicionar barras para Ativo e Cancelado
for status in ["Ativo", "Cancelado"]:
    fig.add_trace(go.Bar(
        x=df_faixas_status.index.astype(str),
        y=df_faixas_status[status],
        name=status
    ))

# Configurar layout do gráfico
fig.update_layout(
    title="Distribuição do Tempo de Permanência dos Clientes (Faturas)",
    xaxis_title="Faixa de Tempo de Permanência (anos)",
    yaxis_title="Quantidade de Clientes (escala logarítmica)",
    barmode="group",  # Barras lado a lado
    yaxis=dict(type="log")  # Aplicar escala logarítmica
)

# Exibir gráfico
fig.show()

# Calcular a porcentagem de cada status dentro de cada faixa de permanência
df_faixas_status_percent = df_faixas_status.div(df_faixas_status.sum(axis=1), axis=0) * 100

df_faixas_status_percent = df_faixas_status_percent.round(2).astype(str) + '%'
# Exibir a tabela de faixas de permanência com status
display(df_faixas_status)
display(df_faixas_status_percent)



C:\Users\gusta\AppData\Local\Temp\ipykernel_18740\2882117830.py:31: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



status_contrato,Ativo,Cancelado
faixa_permanencia,,
0-1 ano,1825,44206
1-2 anos,1626,26308
2-3 anos,856,13563
3-4 anos,877,8976
4-5 anos,750,6466
5-6 anos,970,5039
6-7 anos,613,3864
7-8 anos,1223,3004
8-9 anos,677,2165


status_contrato,Ativo,Cancelado
faixa_permanencia,,
0-1 ano,3.96%,96.04%
1-2 anos,5.82%,94.18%
2-3 anos,5.94%,94.06%
3-4 anos,8.9%,91.1%
4-5 anos,10.39%,89.61%
5-6 anos,16.14%,83.86%
6-7 anos,13.69%,86.31%
7-8 anos,28.93%,71.07%
8-9 anos,23.82%,76.18%


## Tempo de Permanencia - Filtrado - meses_contratos_anos 

In [13]:
import plotly.graph_objects as go
import pandas as pd

# Converter para Pandas para visualização
df_permanencia = sdf_final_filtro.select("meses_contrato_anos", "data_final").toPandas()

# Garantir que há valores para definir o bin máximo
max_anos = df_permanencia["meses_contrato_anos"].max()
bins = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, max_anos + 1]  # Adiciona um espaço extra para o último bin
labels = [
    "0-1 ano",
    "1-2 anos",
    "2-3 anos",
    "3-4 anos",
    "4-5 anos",
    "5-6 anos",
    "6-7 anos",
    "7-8 anos",
    "8-9 anos",
    "9-10 anos",
    "10+ anos"
]

# Criar coluna categórica com os intervalos corrigidos
df_permanencia["faixa_permanencia"] = pd.cut(df_permanencia["meses_contrato_anos"], bins=bins, labels=labels, right=False)

# Classificar contratos como Ativo (nulo) ou Cancelado (não nulo)
df_permanencia["status_contrato"] = df_permanencia["data_final"].apply(lambda x: "Ativo" if pd.isna(x) else "Cancelado")

# Contar a quantidade de contratos ativos e cancelados por faixa de permanência
df_faixas_status = df_permanencia.groupby(["faixa_permanencia", "status_contrato"]).size().unstack(fill_value=0)

# Criar gráfico de barras agrupadas (um do lado do outro)
fig = go.Figure()

# Adicionar barras para Ativo e Cancelado
for status in ["Ativo", "Cancelado"]:
    fig.add_trace(go.Bar(
        x=df_faixas_status.index.astype(str),
        y=df_faixas_status[status],
        name=status
    ))

# Configurar layout do gráfico
fig.update_layout(
    title="Distribuição do Tempo de Permanência dos Clientes (Diferença Datas)",
    xaxis_title="Faixa de Tempo de Permanência (anos)",
    yaxis_title="Quantidade de Clientes (escala logarítmica)",
    barmode="group",  # Barras lado a lado
    yaxis=dict(type="log")  # Aplicar escala logarítmica
)

# Exibir gráfico
fig.show()


# Calcular a porcentagem de cada status dentro de cada faixa de permanência
df_faixas_status_percent = df_faixas_status.div(df_faixas_status.sum(axis=1), axis=0) * 100

df_faixas_status_percent = df_faixas_status_percent.round(2).astype(str) + '%'
# Exibir a tabela de faixas de permanência com status
display(df_faixas_status)
display(df_faixas_status_percent)


C:\Users\gusta\AppData\Local\Temp\ipykernel_18740\3237793005.py:31: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



status_contrato,Ativo,Cancelado
faixa_permanencia,,
0-1 ano,1140,26122
1-2 anos,1018,24917
2-3 anos,801,14915
3-4 anos,871,10885
4-5 anos,723,9027
5-6 anos,1033,7150
6-7 anos,620,5615
7-8 anos,563,4332
8-9 anos,834,3737


status_contrato,Ativo,Cancelado
faixa_permanencia,,
0-1 ano,4.18%,95.82%
1-2 anos,3.93%,96.07%
2-3 anos,5.1%,94.9%
3-4 anos,7.41%,92.59%
4-5 anos,7.42%,92.58%
5-6 anos,12.62%,87.38%
6-7 anos,9.94%,90.06%
7-8 anos,11.5%,88.5%
8-9 anos,18.25%,81.75%


## Tempo de Permanencia/Inicio de Contrato - Filtrado - Faturas

In [14]:
import plotly.graph_objects as go
import pandas as pd
from datetime import datetime

# Converter para Pandas
df_permanencia = sdf_cancelados.select("data_inicio", "faturas_anos").toPandas()

# Converter start_date para datetime
df_permanencia["data_inicio"] = pd.to_datetime(df_permanencia["data_inicio"])

# Criar coluna com o ano do contrato
df_permanencia["ano_inicio"] = df_permanencia["data_inicio"].dt.year

# Obter o ano atual
ano_atual = datetime.today().year

# Calcular estatísticas por ano
estatisticas_por_ano = df_permanencia.groupby("ano_inicio")["faturas_anos"].agg(["median", "mean"]).reset_index()

# Calcular o tempo máximo possível para cada ano (tempo entre ano de início e hoje)
estatisticas_por_ano["tempo_maximo_possivel"] = ano_atual - estatisticas_por_ano["ano_inicio"]

# Calcular a diferença entre o tempo máximo e a mediana
estatisticas_por_ano["diferenca"] = estatisticas_por_ano["tempo_maximo_possivel"] - estatisticas_por_ano["median"]

# Identificar o primeiro ano onde a diferença for menor que um limite (exemplo: 2 anos)
#ano_limite_vies = estatisticas_por_ano.loc[estatisticas_por_ano["diferenca"] < 2, "ano_inicio"].min()

# Criar gráfico interativo
fig = go.Figure()

# Linha da mediana do tempo de permanência por ano
fig.add_trace(go.Scatter(
    x=estatisticas_por_ano["ano_inicio"],
    y=estatisticas_por_ano["median"],
    mode="lines+markers",
    name="Mediana do Tempo de Permanência",
    line=dict(color="blue")
))

# Linha da média do tempo de permanência por ano
fig.add_trace(go.Scatter(
    x=estatisticas_por_ano["ano_inicio"],
    y=estatisticas_por_ano["mean"],
    mode="lines+markers",
    name="Média do Tempo de Permanência",
    line=dict(color="green")
))

# Linha do tempo máximo possível
fig.add_trace(go.Scatter(
    x=estatisticas_por_ano["ano_inicio"],
    y=estatisticas_por_ano["tempo_maximo_possivel"],
    mode="lines",
    name="Tempo Máximo Possível",
    line=dict(color="red", dash="dash")
))



# Layout do gráfico
fig.update_layout(
    title="Detecção do Viés no Tempo de Permanência",
    xaxis_title="Ano de Início do Cliente",
    yaxis_title="Tempo de Permanência (anos)",
    showlegend=True
)

# Exibir gráfico
fig.show()




In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
from datetime import datetime

# Converter para Pandas
df_permanencia = sdf_cancelados.select("data_inicio", "faturas_anos").toPandas()

# Converter start_date para datetime
df_permanencia["data_inicio"] = pd.to_datetime(df_permanencia["data_inicio"])

# Criar coluna com o ano do contrato
df_permanencia["ano_inicio"] = df_permanencia["data_inicio"].dt.year

# Obter o ano atual
ano_atual = datetime.today().year

# Calcular o tempo máximo possível para cada ano
df_permanencia["tempo_maximo_possivel"] = ano_atual - df_permanencia["ano_inicio"]

# Criar Boxplot
fig = px.box(
    df_permanencia,
    x="data_inicio",  # Ano do início do contrato no eixo X
    y="faturas_anos",  # Tempo de permanência no eixo Y
    points="all",  # Mostra todos os pontos para visualizar outliers
    title="Boxplot da Distribuição do Tempo de Permanência por Ano de Início do Contrato",
    labels={"data_inicio": "Ano de Início do Contrato", "faturas_anos": "Tempo de Permanência (anos)"},
    color="data_inicio"  # Diferencia os anos por cor
)

# Adicionar linha guia do tempo máximo possível
fig.add_trace(
    go.Scatter(
        x=df_permanencia["data_inicio"].unique(),
        y=(ano_atual - df_permanencia["data_inicio"].dt.year.unique()),  # Correção
        mode="lines",
        name="Tempo Máximo Possível",
        line=dict(color="red", dash="dash")
    )
)

# Exibir gráfico
fig.show()


## Tempo de Permanencia - Filtrado - meses_contratos_anos 

In [ ]:
import plotly.graph_objects as go
import pandas as pd
from datetime import datetime

# Converter para Pandas
df_permanencia = sdf_cancelados.select("data_inicio", "meses_contrato_anos").toPandas()

# Converter start_date para datetime
df_permanencia["data_inicio"] = pd.to_datetime(df_permanencia["data_inicio"])

# Criar coluna com o ano do contrato
df_permanencia["ano_inicio"] = df_permanencia["data_inicio"].dt.year

# Obter o ano atual
ano_atual = datetime.today().year

# Calcular estatísticas por ano
estatisticas_por_ano = df_permanencia.groupby("ano_inicio")["meses_contrato_anos"].agg(["median", "mean"]).reset_index()

# Calcular o tempo máximo possível para cada ano (tempo entre ano de início e hoje)
estatisticas_por_ano["tempo_maximo_possivel"] = ano_atual - estatisticas_por_ano["ano_inicio"]

# Calcular a diferença entre o tempo máximo e a mediana
estatisticas_por_ano["diferenca"] = estatisticas_por_ano["tempo_maximo_possivel"] - estatisticas_por_ano["median"]

# Identificar o primeiro ano onde a diferença for menor que um limite (exemplo: 2 anos)
#ano_limite_vies = estatisticas_por_ano.loc[estatisticas_por_ano["diferenca"] < 2, "ano_inicio"].min()

# Criar gráfico interativo
fig = go.Figure()

# Linha da mediana do tempo de permanência por ano
fig.add_trace(go.Scatter(
    x=estatisticas_por_ano["ano_inicio"],
    y=estatisticas_por_ano["median"],
    mode="lines+markers",
    name="Mediana do Tempo de Permanência",
    line=dict(color="blue")
))

# Linha da média do tempo de permanência por ano
fig.add_trace(go.Scatter(
    x=estatisticas_por_ano["ano_inicio"],
    y=estatisticas_por_ano["mean"],
    mode="lines+markers",
    name="Média do Tempo de Permanência",
    line=dict(color="green")
))

# Linha do tempo máximo possível
fig.add_trace(go.Scatter(
    x=estatisticas_por_ano["ano_inicio"],
    y=estatisticas_por_ano["tempo_maximo_possivel"],
    mode="lines",
    name="Tempo Máximo Possível",
    line=dict(color="red", dash="dash")
))



# Layout do gráfico
fig.update_layout(
    title="Detecção do Viés no Tempo de Permanência",
    xaxis_title="Ano de Início do Cliente",
    yaxis_title="Tempo de Permanência (anos)",
    showlegend=True
)

# Exibir gráfico
fig.show()




## Churn

In [ ]:
# Contagem total de contratos
total_clientes = sdf_final_filtro.count()

# Contagem de churn (contratos cancelados)
churn_count = sdf_final_filtro.filter(F.col("data_ultimo_boleto").isNotNull()).count()
# Contagem de churn (contratos cancelados)
churn_count_ativos = sdf_final_filtro.filter(F.col("data_ultimo_boleto").isNull()).count()

# Cálculo do churn rate
churn_rate = (churn_count / total_clientes) * 100

# Exibir resultado
print(f"Total de clientes: {total_clientes}")
print(f"Clientes cancelados: {churn_count}")
print(f"Clientes Ativos: {churn_count_ativos}")
print(f"Taxa de Churn: {churn_rate:.2f}%")